In [3]:
import json
from glob import glob

In [4]:
traces = glob("./logs/bigscience/bloom-1b1/**/trace.json", recursive=True)

for trace in traces:
    with open(trace) as f:
        json_dict = json.load(f)

    total_dur = {}
    cpu_op = {}

    for event in json_dict["traceEvents"]:
        if not "cat" in event or not "dur" in event:
            continue
        if event["cat"] in total_dur:
            total_dur[event["cat"]] += event["dur"]
        else:
            total_dur[event["cat"]] = event["dur"]
        if "cpu_op" == event["cat"]:
            if event["name"] in cpu_op:
                cpu_op[event["name"]]["cnt"] += 1
                cpu_op[event["name"]]["dur"] += event["dur"]
            else:
                cpu_op[event["name"]] = {"cnt": 1, "dur": event["dur"]}
    
    for k, v in total_dur.items():
        print(f"{k}: {round(v / 10 ** 6, 3)}s")

    aten_dur = 0
    for i in sorted(cpu_op.items(), key=lambda i: i[1]["cnt"], reverse=True):
        # print(i)
        if "aten::" in i[0]:
            aten_dur += i[1]["dur"]
    print(aten_dur / 10 ** 6, "\n")

cpu_op: 68.05s
user_annotation: 3.471s
cuda_runtime: 21.989s
gpu_memcpy: 0.328s
kernel: 21.49s
gpu_memset: 0.011s
Trace: 23.694s
35.772597 

